<a href="https://colab.research.google.com/github/Tejnium/invoice-scan/blob/master/imagecaptioning)flaskcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install salesforce-lavis
!pip install flask-ngrok
!pip install pickle5

!pip install pyngrok==4.1.1
!ngrok authtoken 2bJ2knQuz9R9g5SkNgmbXsiEzEL_6vjFLjvSfJxXdQFpsY2Ua

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [3]:
# prompt: pip install flask cors

!pip install flask-cors


In [ ]:
import torch
from PIL import Image
import requests

from lavis.models import load_model_and_preprocess
from flask import Flask, request, render_template
from flask_ngrok import run_with_ngrok
import torch
from PIL import Image
import requests
from lavis.models import load_model_and_preprocess
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, vis_processors, _ = load_model_and_preprocess(
    name="blip_caption", model_type="large_coco", is_eval=True, device=device
)


In [ ]:
from flask_cors import CORS
from torchvision.transforms import ToTensor, Resize
from flask import jsonify

app = Flask(__name__)
CORS(app, supports_credentials=True, origins="*")

run_with_ngrok(app)  # Start ngrok when app is run

from flask import jsonify

@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        img_url = request.form.get('img_url')
        raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')
        raw_image = Resize((384, 384))(raw_image)  # Resize the image
        image = ToTensor()(raw_image).unsqueeze(0).to(device)
        captions = model.generate({"image": image}, use_nucleus_sampling=True, num_captions=3)
        return jsonify(captions=captions)
    return "Send a POST request with an image URL to get its captions."

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://dd52-34-75-250-11.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [22/Jan/2024 11:29:42] "POST / HTTP/1.1" 200 -
